In [1]:
from biasanalyzer.api import BIAS
from biasanalyzer.background.threading_utils import BackgroundResult, run_in_background

In [2]:
# create an object of BIAS class
bias = BIAS()

no configuration file specified. Call set_config(config_file_path) next to specify configurations


In [3]:
bias.set_config('/home/hongyi/BiasAnalyzer/config.yaml')

configuration specified in /home/hongyi/BiasAnalyzer/config.yaml loaded successfully


In [4]:
# the configuration file includes root_omop_cdm_database configuration info with an example shown 
# in https://github.com/hyi/HealthDataBias/blob/main/tests/assets/config/test_config.yaml
bias.set_root_omop()

Connected to the OMOP CDM database (read-only).
Cohort Definition table created.
Cohort table created.


In [5]:
# Create baseline cohort result holder
baseline_result = BackgroundResult()

# Start background task to run create_cohort() function for a baseline cohort in a background thread
baseline_thread = run_in_background(
    bias.create_cohort,
    "Young female patients",
    "Young female patients",
    '/home/hongyi/BiasAnalyzer/tests/assets/cohort_creation/test_cohort_creation_condition_occurrence_config_baseline.yaml',
    "system",
    result_holder=baseline_result,
    delay=120  # simulate 5 minutes delay for async testing
)

print("Baseline cohort creation running in background...")


[*] Background task started...
Baseline cohort creation running in background...
template_path: /home/hongyi/BiasAnalyzer/biasanalyzer/sql_templates


Cohort creation:   0%|                                 | 0/3 [00:00<?, ?stage/s]

configuration specified in /home/hongyi/BiasAnalyzer/tests/assets/cohort_creation/test_cohort_creation_condition_occurrence_config_baseline.yaml loaded successfully
Cohort definition inserted successfully.
Cohort Young female patients successfully created.
[DEBUG] Simulating long-running task with 120 seconds delay...
cohort created successfully
[✓] Background task completed.


In [6]:
# Create study cohort result holder
study_result = BackgroundResult()

# Start background task to run create_cohort() function for a study cohort in a background thread
study_thread = run_in_background(
    bias.create_cohort,
    "Young COVID female patients",
    "Young COVID female patients",
    '/home/hongyi/BiasAnalyzer/tests/assets/cohort_creation/test_cohort_creation_condition_occurrence_config_study.yaml',
    "system",
    result_holder=study_result,
    delay=150  # simulate 5 minutes delay for async testing
)

print("Study cohort creation running in background...")

[*] Background task started...
Study cohort creation running in background...


Cohort creation:   0%|                                 | 0/3 [00:00<?, ?stage/s]

configuration specified in /home/hongyi/BiasAnalyzer/tests/assets/cohort_creation/test_cohort_creation_condition_occurrence_config_study.yaml loaded successfully
Cohort definition inserted successfully.
Cohort Young COVID female patients successfully created.
[DEBUG] Simulating long-running task with 150 seconds delay...
cohort created successfully
[✓] Background task completed.


In [11]:
if baseline_result.ready:
    if baseline_result.error:
        print(f"Baseline cohort creation failed: {baseline_result.error}")
    else:
        baseline_cohort = baseline_result.value
        baseline_cohort_def = baseline_cohort.metadata
        print(f"Baseline cohort created with metadata: {baseline_cohort_def}")
        baseline_cohort_data = baseline_cohort.data
        baseline_cohort_stats = baseline_cohort.get_stats()
        print(f"Baseline cohort created with stats: {baseline_cohort_stats}")
else:
    print("Still creating baseline cohort...")

Baseline cohort created with metadata: {'id': 2, 'name': 'Young female patients', 'description': 'Young female patients', 'created_date': datetime.date(2025, 6, 4), 'creation_info': 'WITH ranked_events_condition_occurrence AS ( SELECT person_id, condition_concept_id AS concept_id, condition_start_date AS event_start_date, condition_end_date AS event_end_date, ROW_NUMBER() OVER ( PARTITION BY person_id, condition_concept_id ORDER BY condition_start_date ASC ) AS event_instance FROM condition_occurrence ), ranked_events_drug_exposure AS ( SELECT person_id, drug_concept_id AS concept_id, drug_exposure_start_date AS event_start_date, drug_exposure_end_date AS event_end_date, ROW_NUMBER() OVER ( PARTITION BY person_id, drug_concept_id ORDER BY drug_exposure_start_date ASC ) AS event_instance FROM drug_exposure ), ranked_events_procedure_occurrence AS ( SELECT person_id, procedure_concept_id AS concept_id, procedure_date AS event_start_date, procedure_date AS event_end_date, ROW_NUMBER() OVE

In [12]:
if study_result.ready:
    if study_result.error:
        print(f"Study cohort creation failed: {study_result.error}")
    else:
        study_cohort = study_result.value
        study_cohort_def = study_cohort.metadata
        print(f"Study cohort created with metadata: {study_cohort_def}")
        study_cohort_data = study_cohort.data
        study_cohort_stats = study_cohort.get_stats()
        print(f"Study cohort created with stats: {study_cohort_stats}")
else:
    print("Still creating study cohort...")

Study cohort created with metadata: {'id': 1, 'name': 'Young COVID female patients', 'description': 'Young COVID female patients', 'created_date': datetime.date(2025, 6, 4), 'creation_info': 'WITH ranked_events_condition_occurrence AS ( SELECT person_id, condition_concept_id AS concept_id, condition_start_date AS event_start_date, condition_end_date AS event_end_date, ROW_NUMBER() OVER ( PARTITION BY person_id, condition_concept_id ORDER BY condition_start_date ASC ) AS event_instance FROM condition_occurrence ), domain_qualifying_events AS ( (SELECT person_id, event_start_date, event_end_date FROM ranked_events_condition_occurrence WHERE concept_id = 37311061) ), filtered_cohort AS ( SELECT c.person_id, MIN(c.event_start_date) AS cohort_start_date, MAX(c.event_end_date) AS cohort_end_date FROM domain_qualifying_events c JOIN person p ON c.person_id = p.person_id WHERE 1=1 AND p.gender_concept_id = 8532 AND p.year_of_birth >= 2000 AND p.year_of_birth <= 2020 GROUP BY c.person_id ) SELE

In [13]:
# compare the baseline and user study cohorts
if baseline_result.ready and study_result.ready:
    print(f"first 5 patient in baseline cohort data: {baseline_cohort_data[:5]}")
    print(f"first 5 patient in study cohort data: {study_cohort_data[:5]}")
    baseline_cohort_age_stats = baseline_cohort.get_stats("age")
    print(f'the baseline cohort age stats: {baseline_cohort_age_stats}')
    baseline_cohort_gender_stats = baseline_cohort.get_stats("gender")
    print(f'the baseline cohort gender stats: {baseline_cohort_gender_stats}')
    study_cohort_age_stats = study_cohort.get_stats("age")
    print(f'the study cohort age stats: {study_cohort_age_stats}')
    study_cohort_gender_stats = study_cohort.get_stats("gender")
    print(f'the study cohort gender stats: {study_cohort_gender_stats}')
    result = bias.compare_cohorts(1, 2)
    print(result)

first 5 patient in baseline cohort data: [{'subject_id': 42583, 'cohort_definition_id': 2, 'cohort_start_date': datetime.date(2020, 4, 26), 'cohort_end_date': datetime.date(2020, 5, 12)}, {'subject_id': 33685, 'cohort_definition_id': 2, 'cohort_start_date': datetime.date(2017, 12, 8), 'cohort_end_date': datetime.date(2020, 5, 10)}, {'subject_id': 74383, 'cohort_definition_id': 2, 'cohort_start_date': datetime.date(2019, 1, 31), 'cohort_end_date': datetime.date(2020, 3, 25)}, {'subject_id': 23986, 'cohort_definition_id': 2, 'cohort_start_date': datetime.date(2019, 6, 15), 'cohort_end_date': datetime.date(2020, 3, 28)}, {'subject_id': 93962, 'cohort_definition_id': 2, 'cohort_start_date': datetime.date(2019, 7, 1), 'cohort_end_date': datetime.date(2020, 5, 15)}]
first 5 patient in study cohort data: [{'subject_id': 53949, 'cohort_definition_id': 1, 'cohort_start_date': datetime.date(2020, 2, 28), 'cohort_end_date': datetime.date(2020, 3, 11)}, {'subject_id': 22344, 'cohort_definition_id'

In [14]:
bias.cleanup()

Connection to BiasDatabase closed.
Connection to the OMOP CDM database closed.
